# Importing Data
In this notebook we well donwload data from different sources orgnise it and store it in our local environnement 

In [1]:
from datasets import load_dataset
import pandas as pd
import time 

d:\Users\chatbot_emotions_detection\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Arabic Emotions Datasets

|label | Label description | Count | 
|------|-------------------|-------| 
|0     |     none          | 1550  | 
|1     |    anger          | 1444  | 
|2     |joy                |  1281 | 
|3     |sadness            | 1256  | 
|4     |love               | 1220  | 
|5     |sympathy           | 1062  | 
|6     |surprise           | 1045  | 
|7     |fear               | 1207  |

In [3]:
#Importing data
emotions_ar = load_dataset ( 'emotone_ar' )
emotions_ar

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 10065
    })
})

In [5]:
emotions_ar_train = emotions_ar ['train']
emotions_ar_train

Dataset({
    features: ['tweet', 'label'],
    num_rows: 10065
})

In [8]:
#Structuring data
df_emotions_arabic = pd.DataFrame()
df_emotions_arabic['tweet'] = emotions_ar_train['tweet']
df_emotions_arabic['label'] = emotions_ar_train['label']
df_emotions_arabic

,tweet,label
0,الاوليمبياد الجايه هكون لسه ف الكليه ..,0
1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,1
2,كتنا نيله ف حظنا الهباب xD,3
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,2
4,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,0
...,...,...
10060,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,3
10061,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,6
10062,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,0
10063,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,2


In [13]:
#Storing data
df_emotions_arabic.to_excel ( 'Datasets/emtions_detection_datasets/text_emotion_egypt.xlsx' )

### Translate an Emotions Dataset

In [2]:
#Packages 
from transformers import pipeline 
import sentencepiece

In [3]:
#Pipeline translation and model
#MODEL_NAME = "Helsinki-NLP/opus-mt-en-ar"          # BLUE = 14.0
MODEL_NAME = "Helsinki-NLP/opus-mt-tc-big-en-ar"    #BLUE = 29.0
#MODEL_NAME = 'marefa-nlp/marefa-mt-en-ar'          #BLUE = ? 
translator = pipeline('translation', model= MODEL_NAME)

d:\Users\chatbot_emotions_detection\venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
#Sample text for testing
text = "Hello, what is the emotion behind this text: I am happy but at same time sad. I dont realy understand my feelings "

translator ( text )

[{'translation_text': 'مرحبا، ما هي العاطفة وراء هذا النص: أنا سعيد ولكن في نفس الوقت حزين. أنا لا أفهم حقا مشاعري'}]

The model works fine 

We will start applying it on our english data

In [6]:
#English data
text_emotion_eng =  pd.read_excel ( 'D:/Users/chatbot_emotions_detection/Datasets/emtions_detection_datasets/02_text_emotion_eng_dair_ai_emotion.xlsx')
text_emotion_eng

,text,label,emotion
0,i feel awful about it too because it s my job ...,0,sadness
1,im alone i feel awful,0,sadness
2,ive probably mentioned this before but i reall...,1,joy
3,i was feeling a little low few days back,0,sadness
4,i beleive that i am much more sensitive to oth...,2,love
...,...,...,...
416804,that was what i felt when i was finally accept...,1,joy
416805,i take every day as it comes i m just focussin...,4,fear
416806,i just suddenly feel that everything was fake,0,sadness
416807,im feeling more eager than ever to claw back w...,1,joy


In [7]:
#Extracting english sentences 
content = text_emotion_eng ['text']
content

0         i feel awful about it too because it s my job ...
1                                     im alone i feel awful
2         ive probably mentioned this before but i reall...
3                  i was feeling a little low few days back
4         i beleive that i am much more sensitive to oth...
                                ...                        
416804    that was what i felt when i was finally accept...
416805    i take every day as it comes i m just focussin...
416806        i just suddenly feel that everything was fake
416807    im feeling more eager than ever to claw back w...
416808    i give you plenty of attention even when i fee...
Name: text, Length: 416809, dtype: object

In [29]:
#Méthode traduction phrase par phrase
"""
sentences = content[0: 50]
total_sentences = len (sentences)
processed_sentences = 0
fr_sentences = []

for sentence in sentences:
    fr_sentences.append (translator ( sentence  ))
    processed_sentences += 1
    # Calculate the progress percentage
    progress = processed_sentences / total_sentences * 100
    # Display the progress
    print(f"Progress: {progress:.2f}%", end="\r")
    # Optionally, you can add some delay to simulate processing time
    time.sleep(0.1)
"""

La méthode traduction phrase après phrase prend 2 fois plus de temps que la méthode translate par batch.

C'est pourquoi nous allons procédé par la méthode batch

This méthod should be repeated as much as needed, starting from [0 - 1000] batch till 40,000

In [8]:
#Defining translate_en_fr_emotion function that translate english batch of sentences to french.

def translate_en_ar_emotion (batch, translator, sentiments):
    """
    This function takes a batch of english sentences and translate them using translator then concatinate them
    with corresponding sentiments from text_emotion_eng dataset.
    """
    # transfoming df columns to a list of items
    batch = batch.to_list()
    sentiments = sentiments.to_list()
    
    #translating the batch
    fr_batch = translator(batch)
    #Transform sentences to a list 
    sentences = []
    for i in range (len(fr_batch)):
        sentences.append (fr_batch[i]['translation_text'])
    sentences
    #Concatinating translated sentences with their corresponding sentiments
    text_emotion_french = pd.DataFrame()
    text_emotion_french['texte'] = sentences
    text_emotion_french['emotion'] = sentiments

    return text_emotion_french

In [30]:
#Translating the first batch
start = 0
end = 500
batch0000_500= translate_en_ar_emotion ( content [start: end], translator, text_emotion_eng['emotion'][start: end]  )

In [36]:
#Translating the first batch
start = 500
end = 1000
batch500_1000= translate_en_ar_emotion ( content [start: end], translator, text_emotion_eng['emotion'][start: end]  )

In [37]:
#Translating the first batch
start = 1000
end = 2000
batch1000_2000= translate_en_ar_emotion ( content [start: end], translator, text_emotion_eng['emotion'][start: end]  )

In [45]:
#Translating the first batch
start = 2000
end = 3000
batch2000_3000= translate_en_ar_emotion ( content [start: end], translator, text_emotion_eng['emotion'][start: end]  )

In [39]:
#Translating the first batch
start = 3000
end = 4000
batch3000_4000= translate_en_ar_emotion ( content [start: end], translator, text_emotion_eng['emotion'][start: end]  )

In [52]:
#Translating the first batch
start = 5000
end = 6000
batch5000_6000= translate_en_ar_emotion ( content [start: end], translator, text_emotion_eng['emotion'][start: end]  )

In [57]:
#Translating the first batch
start = 6000
end = 8000
batch6000_8000= translate_en_ar_emotion ( content [start: end], translator, text_emotion_eng['emotion'][start: end]  )

In [54]:
#Translating the first batch
start = 8000
end = 10000
batch8000_10000= translate_en_ar_emotion ( content [start: end], translator, text_emotion_eng['emotion'][start: end]  )

In [55]:
#Concatinating data
batch0000_10000 = pd.concat ( [ batch0000_500, batch500_1000, batch1000_2000, batch2000_3000, batch3000_4000, batch4000_5000, batch5000_6000, batch8000_10000 ] )
batch0000_10000.reset_index ( drop = True, inplace= True )
batch0000_10000

,texte,emotion
0,أشعر بالسوء حيال ذلك أيضًا لأن وظيفتي هي جعله ...,sadness
1,أنا وحدي أشعر بفظاعة,sadness
2,ربما ذكرت هذا من قبل ولكن أنا حقا لا أشعر بالف...,joy
3,كنت أشعر قليلا منخفضة بضعة أيام إلى الوراء,sadness
4,أنا أؤمن بأنني أكثر حساسية لمشاعر الشعوب الأخر...,love
...,...,...
7995,أنا خائف وأشعر بالحرج وفي بعض الأحيان حتى الاك...,sadness
7996,كنت في أدنى وزني الوزن الذي ترك لي الشعور بالب...,anger
7997,أشعر أنه يجب أن تكون في حياتي ما زلت أشك في ذلك,fear
7998,شربت القهوة مع صديق كاتب قبل أسبوعين من صديق ي...,anger


In [56]:
#Saving the batch
batch0000_10000.to_excel ( "D:/Users/chatbot_emotions_detection/Datasets/emtions_detection_datasets/text_emotion_arabic_batches/batch0000_10000.xlsx" )

In [11]:
#Translating the first batch
start = 0
end = 5000
fr_batch0000_5000= translate_en_fr_emotion ( content [start: end], translator, text_emotion_eng['emotion'][start: end]  )

In [34]:
#storing
#fr_batch0000_5000.to_excel ('D:/Users/chatbot_emotions_detection/Datasets/emtions_detection_datasets/text_emotion_french_batches/fr_batch_0000_5000.xlsx')

In [9]:
#Translating batches
start = 5000
end = 7000
fr_batch5000_7000= translate_en_fr_emotion ( content [start: end], translator, text_emotion_eng['emotion'][start: end]  )

In [12]:
#Storing the data
fr_batch5000_7000.to_excel ('D:/Users/chatbot_emotions_detection/Datasets/emtions_detection_datasets/text_emotion_french_batches/fr_batch_5000_7000.xlsx')

In [ ]:
#Translating batches
start = 7000
end = 10000
fr_batch7000_10000= translate_en_fr_emotion ( content [start: end], translator, text_emotion_eng['emotion'][start: end]  )

In [108]:
#Translating batches
start = 15000
end = 20000
fr_batch15000_20000= translate_en_fr_emotion ( content [start: end], translator, text_emotion_eng['sentiment'][start: end]  )

In [110]:
#Translating batches
start = 20000
end = 25000
fr_batch20000_25000= translate_en_fr_emotion ( content [start: end], translator, text_emotion_eng['sentiment'][start: end]  )

In [118]:
#Translating batches
start = 25000
end = 30000
fr_batch25000_30000= translate_en_fr_emotion ( content [start: end], translator, text_emotion_eng['sentiment'][start: end]  )

In [115]:
#Concatinating the batches in one dataset
fr_batch0000_30000 = pd.concat ([ fr_batch0000_5000, fr_batch5000_10000, fr_batch10000_15000, fr_batch15000_20000, fr_batch20000_25000, fr_batch25000_30000])

In [125]:
#Storing the data
fr_batch0000_30000.to_excel ('Datasets/emtions_detection_datasets/text_emotion_french_batches/fr_batch_0000_30000.xlsx')

In [139]:
#Storing data in the main folder
fr_batch0000_30000.to_excel ('Datasets/emtions_detection_datasets/text_emotion_french.xlsx', index = False)

### English dataset 

In [5]:
#Importing data
data = pd.read_json ( 'D:/Users/chatbot_emotions_detection/Datasets/emtions_detection_datasets/text_emotion_eng_2_go_emotion.jsonl', lines = True )
data

,text,label
0,i feel awful about it too because it s my job ...,0
1,im alone i feel awful,0
2,ive probably mentioned this before but i reall...,1
3,i was feeling a little low few days back,0
4,i beleive that i am much more sensitive to oth...,2
...,...,...
416804,that was what i felt when i was finally accept...,1
416805,i take every day as it comes i m just focussin...,4
416806,i just suddenly feel that everything was fake,0
416807,im feeling more eager than ever to claw back w...,1


Labels : sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).

Adding the correspondant emotion to data

In [7]:
#emotion mapping
emotion_mapping = {0: 'sadness',1: 'joy', 2:'love', 3: 'anger', 4: 'fear', 5: 'surprise' }
emotion_mapping


{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [8]:
#Getting emotion form label
emotions = []
for label in data['label']:
    emotions.append(emotion_mapping [label])

In [10]:
len (emotions)

416809

In [11]:
# addind emotions to data
data['emotion'] = emotions
data

,text,label,emotion
0,i feel awful about it too because it s my job ...,0,sadness
1,im alone i feel awful,0,sadness
2,ive probably mentioned this before but i reall...,1,joy
3,i was feeling a little low few days back,0,sadness
4,i beleive that i am much more sensitive to oth...,2,love
...,...,...,...
416804,that was what i felt when i was finally accept...,1,joy
416805,i take every day as it comes i m just focussin...,4,fear
416806,i just suddenly feel that everything was fake,0,sadness
416807,im feeling more eager than ever to claw back w...,1,joy


In [12]:
#Saving the data
#data.to_excel ( 'D:/Users/chatbot_emotions_detection/Datasets/emtions_detection_datasets/text_emotion_eng_2_go_emotion.xlsx', index = False )